# Save sample data to SQLite database
I choose to access the data from a SQLite database for its simplicity and ease of use. 

When flexibility becomes more important, SQLAlchemy could be used as a wrapper, so the code can be easily abstracted from the underlying database.

First we need a function to create the two database tables CC050 and CI050, respectively. 
Column names, its data types and example data are taken from the provided specific_example.txt. 
The rows of the tables are represented as tuples and the data of the tables as lists of tuples.

In [11]:
import sqlite3


def create_tables(conn):
    """Create tables CC050 and CI050 in the SQLite database."""
    with conn:
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS CC050 (
                        date TEXT,
                        clearing_member TEXT,
                        account TEXT,
                        margin_type TEXT,
                        margin REAL
                        )''')

        cursor.execute('''CREATE TABLE IF NOT EXISTS CI050 (
                        date TEXT,
                        time_of_day TEXT,
                        clearing_member TEXT,
                        account TEXT,
                        margin_type TEXT,
                        margin REAL
                        )''')

# Example data from specific_example.txt file for EoD (end of day) reports CC050 table
cc050_data = [
        ('2020-05-11', 'Bank 1', 'A1', 'SPAN', 3212.2),
        ('2020-05-11', 'Bank 1', 'A1', 'IMSM', 837.1),
        ('2020-05-11', 'Bank 1', 'A2', 'SPAN', 8963.3),
        ('2020-05-11', 'Bank 1', 'A2', 'IMSM', 76687.9),
        ('2020-05-11', 'Bank 2', 'A1', 'SPAN', 821.4),
        ('2020-05-11', 'Bank 2', 'A1', 'SPAN', 8766.4),
]

# Same for Intraday reports CI050 table
ci050_data = [
        ('2020-05-11', '18:00:00', 'Bank 1', 'A1', 'SPAN', 2882.2),
        ('2020-05-11', '18:00:00', 'Bank 1', 'A1', 'IMSM', 988.1),
        ('2020-05-11', '18:00:00', 'Bank 1', 'A2', 'SPAN', 788.3),
        ('2020-05-11', '18:00:00', 'Bank 1', 'A2', 'IMSM', 908.9),
        ('2020-05-11', '18:00:00', 'Bank 2', 'A1', 'SPAN', 123.4),
        ('2020-05-11', '18:00:00', 'Bank 2', 'A1', 'IMSM', 8326.4),
        ('2020-05-11', '19:00:00', 'Bank 1', 'A1', 'SPAN', 3212.2),
        ('2020-05-11', '19:00:00', 'Bank 1', 'A1', 'IMSM', 837.1),
        ('2020-05-11', '19:00:00', 'Bank 1', 'A2', 'SPAN', 8963.3),
        ('2020-05-11', '19:00:00', 'Bank 1', 'A2', 'IMSM', 76687.9),
        ('2020-05-11', '19:00:00', 'Bank 2', 'A1', 'SPAN', 821.4),
        ('2020-05-11', '19:00:00', 'Bank 2', 'A1', 'IMSM', 8766.4),
        ('2020-05-12', '08:00:00', 'Bank 1', 'A1', 'SPAN', 3212.2),
        ('2020-05-12', '08:00:00', 'Bank 1', 'A1', 'IMSM', 837.1),
        ('2020-05-12', '08:00:00', 'Bank 1', 'A2', 'SPAN', 8963.3),
        ('2020-05-12', '08:00:00', 'Bank 1', 'A2', 'IMSM', 76687.9),
        ('2020-05-12', '08:00:00', 'Bank 2', 'A1', 'SPAN', 821.4),
        ('2020-05-12', '08:00:00', 'Bank 2', 'A1', 'IMSM', 8766.4),
        ('2020-05-12', '09:00:00', 'Bank 1', 'A1', 'SPAN', 3133.9),
        ('2020-05-12', '09:00:00', 'Bank 1', 'A1', 'IMSM', 137.1),
        ('2020-05-12', '09:00:00', 'Bank 1', 'A2', 'SPAN', 2963.3),
        ('2020-05-12', '09:00:00', 'Bank 1', 'A2', 'IMSM', 74687.9),
        ('2020-05-12', '09:00:00', 'Bank 2', 'A1', 'SPAN', 811.4),
        ('2020-05-12', '09:00:00', 'Bank 2', 'A1', 'IMSM', 8366.4),
]

We need a function to insert the data from the objects created above into the database. 
The function is flexible in regard of column number, since CC050 and CI050 differ in number of columns.

In [12]:
def insert_data(conn, table_name, data):
    """Insert example data provided into the specified table."""
    with conn:
        cursor = conn.cursor()
        placeholders = ', '.join(['?'] * len(data[0]))
        query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.executemany(query, data)


In the last step this function is used to insert the respective data into its tables. 
The database "LZDB_dummy.db" is created in the repository root folder and will be accessed for the validation functionality.

In [14]:
def populate_database(database_path, cc050_data, ci050_data):
    """Populate the SQLite database with example data provided."""
    with sqlite3.connect(database_path) as conn:
        create_tables(conn)
        insert_data(conn, 'CC050', cc050_data)
        insert_data(conn, 'CI050', ci050_data)

populate_database("../LZDB_dummy.db", cc050_data, ci050_data)


In [ ]:
Make sure to run the last cell only once. Otherwise the database will contain dublicates.

# Margin validation functionality
Before running the main() program to validate the margins, we write several functions that implement the functionalities such as logging, fetching the data from the database, the validation logic and the alert mail sending.

## Logging
We use the logging library and configure filename, logging level and format.

In [43]:
import logging

# Setup logging
logging.basicConfig(filename='../margin_validation.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %('
                                                                                     'message)s')


## Access database content 
We define fetch_margins function. 
It queries the margin data for a given table name, report date and (in case of table CI050) report time. 
Since CI050 and CC050 differ in column number, the parameter report_time is used to distinguish two cases of querrying. The function returns lists of tuples, the latter being the representation of a row in the table.

In [44]:
def fetch_margins(conn, table_name, report_date, report_time=None):
    """Fetch rows from the specified table given the report date and (optionally) report time.
       RETURN TYPE: List of Tuples"""
    cursor = conn.cursor()
    if report_time:
        cursor.execute(
            f'SELECT date, time_of_day, clearing_member, account, margin_type, margin FROM {table_name} WHERE date = '
            f'? AND time_of_day = ?',
            (report_date, report_time))
    else:
        cursor.execute(f"SELECT date, clearing_member, account, margin_type, margin FROM {table_name} WHERE date = ?",
                       (report_date,))
    margins_list = cursor.fetchall()
    return margins_list


## Logic functionality
The function validate_elements_of_list1_in_list2(list1, list2) 
will be central to the validation logic. 
It takes two lists of tuples as parameters. 
It checks for each element of the first list if it is also part of the second list. 
The last four elements of the tuple of the first list must equal the respective last four 
elements of the tuple of the second list. 
Each element of list 1 that is not contained in list 2 is saved to discrepancies. 
When the function finishes the list discrepancies is returned. 
I.e. if all elements of the first list are contained in the second, "None" is returned.

In [45]:
def validate_elements_of_list1_in_list2(list1, list2):
    """
    Check for two lists of tuples if all the first lists elements are found in list two.
    For the comparison irrelevant columns date and (optionally) time_of_day are sliced of.
    """

    # note: only the four last columns are relevant to compare cc050_data and ci050_data, respectively.
    #       we keep columns date, time_of_day for error message / logging.
    discrepancies = []
    for list1_row in list1:
        list1_relevant_rows = list1_row[-4:]
        found_match = False
        for list2_row in list2:
            list2_relevant_rows = list2_row[-4:]
            if list1_relevant_rows == list2_relevant_rows:
                found_match = True
                break
        if not found_match:
            discrepancies.append(f"Margin missing: {list1_row}")
    return discrepancies


## Alert mail sending
Mails to multiple receivers can be sent by using the Simple Mail Transfer Protocoll (SMTP).
A mail server has a be able to support that protocol and must be configured properly. 

We varified the functionality using Gmail and two private Gmail adresses. 
Since credentials such as email adresses (and passwords!) should not be found in public repositories,
the respective data is imported from a local file that is not part of the repository.
To use the send mail functionality, specify the data for senders and receivers as shown in  credentials_example.py.

Note: Another option would be using environment variables to store secrets. 

In [46]:
import smtplib

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import sys
sys.path.append("..")
import credentials


def send_email(subject, body):
    """Sends an alert email when validation failed."""

    # email credentials NOT shared in public repo. get them from credentials file
    sender_email = credentials.sender_email
    sender_email_pw = credentials.sender_email_pw
    receiver_email = credentials.receiver_email

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = ", ".join(receiver_email)
    message["Subject"] = subject

    # Structure email, add body to email
    message.attach(MIMEText(body, "plain"))

    # Convert the message to a string and send it
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()

        server.login(sender_email, sender_email_pw)
        text = message.as_string()
        server.sendmail(sender_email, receiver_email, text)


## Queries for first and last report time of a specific data
For the validation program to be flexible regarding the first and last report time of a given report date,
a helper function is defined.
The function get_min_time_of_day and get_max_time_of_day return the minimal and maximal entries in time_of_day column, respectively. 

In [47]:
def get_min_time_of_day(conn, table_name, report_date):
    """Get the minimal time of day for a given date from the specified table."""
    cursor = conn.cursor()
    cursor.execute(f"SELECT MIN(time_of_day) FROM {table_name} WHERE date = ?", (report_date,))
    min_time = cursor.fetchone()[0]
    return min_time


def get_max_time_of_day(conn, table_name, report_date):
    """Get the maximal time of day for a given date from the specified table."""
    cursor = conn.cursor()
    cursor.execute(f"SELECT MAX(time_of_day) FROM {table_name} WHERE date = ?", (report_date,))
    max_time = cursor.fetchone()[0]
    return max_time


## Execute the validation
First a remark regarding the dates. The sample set has a specific range of dates.
That's why the variables today and yesterday are hardcode to '2020-05-12' and  '2020-05-11', respectively. 
In production they will be variables delivered by the datetime library (see code comments).

###Validation logic
We must check that yesterdays CC050 margins match with yesterdays last as well as todays first CI050 margins as a one-to-one correspondence. The relations are be invertible and we check the inverse as well. In total four checks are done. 

After the checks duplicates of missing margins are removed.

Finally a success message is printed to stdout in the case of no error occuring.
In case of erroneous data and failing validation checks, 
the respective error messages are printed to stdout, 
a Logfile and sent as alert mails to stakeholders.

Note: If no credentials for email sender and receiver are given, 
this main() will fail. To deactivate sending alert mails, remove the two send_email() commands.

In [48]:
#import datetime

def main():
    conn = sqlite3.connect('../LZDB_dummy.db')

    # we suppose the checks are run at 10am on
    #today = datetime.date.today().strftime('%Y-%m-%d')
    today = '2020-05-12'
    # therefore...
    #yesterday = today - datetime.timedelta(days=1)
    #yesterday = yesterday.strftime('%Y-%m-%d')
    yesterday = '2020-05-11'

    # CHECK 1: Check, if the end-of-day values of the previous day are the same as the first intraday values.

    cc050_yesterday = fetch_margins(conn, 'CC050', yesterday)
    ci050_today_first = fetch_margins(conn, 'CI050', today,
                                      report_time=get_min_time_of_day(conn, 'CI050', today))

    # Validate that yesterdays eod cc margins are found in today's first ci margin. And vice versa.
    missing_cc050 = validate_elements_of_list1_in_list2(cc050_yesterday, ci050_today_first)
    missing_ci050 = validate_elements_of_list1_in_list2(ci050_today_first, cc050_yesterday)

    # CHECK 2: Check if the end-of-day values are the same as the last intraday values.

    ci050_yesterday_last = fetch_margins(conn, 'CI050', yesterday,
                                         report_time=get_max_time_of_day(conn, 'CI050', yesterday))

    missing_cc050 += validate_elements_of_list1_in_list2(cc050_yesterday, ci050_yesterday_last)
    missing_ci050 += validate_elements_of_list1_in_list2(ci050_yesterday_last, cc050_yesterday)

    # Remove duplicates
    missing_cc050 = list(set(missing_cc050))
    missing_ci050 = list(set(missing_ci050))

    # If check1 or check2 failed write error messages, add them to the log file and send alert emails
    if not missing_cc050 and not missing_ci050:
        print("Margins validation successful. No discrepancies found.\n")
    if missing_cc050:
        error_message = "\nValidation failed! Missing margins in CC050:\n"
        for error in missing_cc050:
            error_message += error + '\n'
        print(error_message)
        logging.error(error_message)
        send_email("Validation Alert: Missing Margins in CC050", error_message)
    if missing_ci050:
        error_message = "\nValidation failed! Missing margins in CI050:\n"
        for error in missing_ci050:
            error_message += error + '\n'
        print(error_message)
        logging.error(error_message)
        send_email("Validation Alert: Missing Margins in CI050", error_message)

    conn.close()


if __name__ == '__main__':
    main()



Validation failed! Missing margins in CC050:
Margin missing: ('2020-05-11', 'Bank 2', 'A1', 'SPAN', 8766.4)


Validation failed! Missing margins in CI050:
Margin missing: ('2020-05-12', '08:00:00', 'Bank 2', 'A1', 'IMSM', 8766.4)
Margin missing: ('2020-05-11', '19:00:00', 'Bank 2', 'A1', 'IMSM', 8766.4)

